In [81]:
import pandas as pd
import numpy as np


In [83]:
data_trade = pd.read_excel('data_trade.xlsx')
data_trade = data_trade.set_axis(['State', 'Money from Exports', '% of GDP', 'Largest Market', 'Largest Export Product'], axis=1)


data_carbon = pd.read_excel('data_carbon.xlsx')
data_carbon['%'] = data_carbon['%'] * 100
combined_data = pd.merge(data_trade, data_carbon, on='State')

KeyError: '%'

In [70]:
combined_data.rename(columns={"% of GDP" : 'Percent of State GDP'})
combined_data.to_csv('combined_data.csv', index=False)
combined_data

,State,Money from Exports,% of GDP,Largest Market,Largest Export Product,%,CO2,CO2 (tons per capita)
0,California,1.781811e+11,4.80,Mexico,computer & electronic products,6.6,303.7,7.7
1,New York,8.168320e+10,13.50,Canada,miscellaneous manufactured commodities,3.1,143.7,7.1
2,Washington,7.796822e+10,25.90,China,transportation equipment,1.5,68.3,8.9
3,Louisiana,6.729707e+10,7.40,Mexico,petroleum & coal products,4.0,183.6,39.4
4,Illinois,6.549141e+10,10.80,Canada,"machinery, except electrical",3.7,170.2,13.3
5,Michigan,5.803477e+10,5.40,Canada,transportation equipment,3.0,136.9,13.6
6,Florida,5.723664e+10,7.90,Brazil,computer & electronic products,4.5,207.3,9.6
7,Ohio,5.440383e+10,5.10,Canada,transportation equipment,4.0,185.8,15.7
8,Pennsylvania,4.119264e+10,6.70,Canada,chemicals,4.2,193.4,14.9
9,Georgia,4.061332e+10,10.60,Canada,transportation equipment,2.5,116.4,10.9


In [74]:
cut_data = combined_data[['State', 'Money from Exports', '% of GDP', 'Largest Export Product', 'CO2']]

In [75]:
cut_data['Percent GDP per CO2'] = cut_data['% of GDP'] / cut_data['CO2']

C:\Users\drago\AppData\Local\Temp\ipykernel_8372\2401093912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cut_data['Percent GDP per CO2'] = cut_data['% of GDP'] / cut_data['CO2']


In [77]:
cut_data.sort_values(by=['Percent GDP per CO2'], ascending=False)

,State,Money from Exports,% of GDP,Largest Export Product,CO2,Percent GDP per CO2
45,District of Columbia,2.724646e+09,3.90,transportation equipment,2.4,1.625000
50,Hawaii,6.597746e+08,18.80,petroleum & coal products,15.0,1.253333
51,American Samoa,3.320000e+07,0.32,Animal meal and pellets,0.4,0.800000
43,Vermont,2.919968e+09,4.30,computer & electronic products,5.4,0.796296
38,New Hampshire,5.306051e+09,8.70,computer & electronic products,12.4,0.701613
40,Delaware,4.713639e+09,5.10,chemicals,12.4,0.411290
2,Washington,7.796822e+10,25.90,transportation equipment,68.3,0.379209
46,Rhode Island,2.406562e+09,3.30,waste and scrap,9.8,0.336735
23,Puerto Rico,1.806440e+10,6.20,chemicals,19.0,0.326316
34,Nebraska,7.952229e+09,12.20,food manufactures,46.5,0.262366
